In [ ]:
import os
import glob
import cv2
import re
import Levenshtein
from tqdm import tqdm

In [ ]:
root_dir = "."
non_shadow_paths = sorted(glob.glob("../datasets/OSR/shadowfree_image/*")) # output data path
shadow_paths = sorted(glob.glob("../datasets/OSR/shadowfree_image/*"))
paths = {"non_shadow": non_shadow_paths, "shadow": shadow_paths}

In [ ]:
right_rot = ["001", "002", "017", "019", "020", "021", "023"]
reedable = ["003", "005", "006", "007", "008", "009", "010", "011", "012", "013", "014", "015", "016", "024", "025", "026"]
if not os.path.exists("ocr/hocr"):
    os.mkdir("ocr/hocr")
if not os.path.exists("ocr/ocr"):
    os.mkdir("ocr/ocr")
if not os.path.exists("ocr/ocr_text"):
    os.mkdir("ocr/ocr_text")

for shadow_type in ["shadow", "non_shadow"]:
    for p in tqdm(paths[shadow_type]):
        file_name = p.split("/")[-1].split(".")[0]
        if not file_name.split("_")[0] in reedable:
            continue
        os.system(f"tesseract {p} ocr/hocr/{shadow_type}_{file_name} hocr")
        os.system(f"tesseract {p} ocr/ocr_text/{shadow_type}_{file_name}")

        img = cv2.imread(p)
        if file_name.split("_")[0] in right_rot:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        
        with open(f"ocr/hocr/{shadow_type}_{file_name}.hocr") as f:
            lines = f.readlines()

            for line in lines:
                e = re.findall("ocrx_word|bbox|[0-9]+|x_wconf|>.+<", line)
                if "x_wconf" in e:
                    index = e.index("bbox")
                    conf_idx = e.index("x_wconf")
                    pt = [int(ax) for ax in e[index+1:index+5]]
                    conf = int(e[conf_idx+1])
                    text = e[-1][1:-1]

                    cv2.rectangle(img, pt1=(pt[0], pt[1]), pt2=(pt[2], pt[3]), color=(0, 255, 0), thickness=2)
                    cv2.putText(img, text, org=(pt[0], pt[1]), fontFace=cv2.FONT_HERSHEY_PLAIN, color=(255, 0, 0), fontScale=1)
        cv2.imwrite(f"ocr/ocr/{shadow_type}_{file_name}.png", img)


In [ ]:
distances = []
shadow_text_path = sorted(glob.glob("ocr/ocr_text/shadow_*.txt"))
non_shadow_text_path = sorted(glob.glob("ocr/ocr_text/non_shadow_*.txt"))
for shadow_p, non_shadow_p in zip(shadow_text_path, non_shadow_text_path):
        file_name = shadow_p.split("/")[-1].split(".")[0].split("_")[-1]

        shadow_f = open(shadow_p)
        shadow_text = "".join(map(lambda x: x.strip(), shadow_f.readlines()))
        shadow_f.close()

        non_shadow_f = open(non_shadow_p)
        non_shadow_text = "".join(map(lambda x: x.strip(), non_shadow_f.readlines()))
        non_shadow_f.close()

        distance = Levenshtein.distance(shadow_text, non_shadow_text)

        if distance == 0:
                print("file name:", file_name)
                print(shadow_text)
                print(non_shadow_text)

        distances.append(distance)

        shadow_f.close()
        non_shadow_f.close()

print(sum(distances)/len(shadow_text_path))